In [4]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2


In [8]:
raw_audio_dir = "E:\Music Classifier\Scratch\Data\genres_original"
spectrogram_dir = "E:\Music Classifier\Scratch\Data\spectrograms"
csv_dir = r"E:\Music Classifier\Scratch\Data\features.csv"

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
C:\Users\ebube\AppData\Local\Temp\ipykernel_8812\3342716040.py:1: SyntaxWarning: invalid escape sequence '\M'
  raw_audio_dir = "E:\Music Classifier\Scratch\Data\genres_original"
C:\Users\ebube\AppData\Local\Temp\ipykernel_8812\3342716040.py:2: SyntaxWarning: invalid escape sequence '\M'
  spectrogram_dir = "E:\Music Classifier\Scratch\Data\spectrograms"


In [18]:
def generate_mel_spectrogram(audio_path, image_save_path, sr=22050, n_fft=2048, hop_length=512, n_mels=128, top_db=80, cmap='magma'):
    try:
        y, sr = librosa.load(audio_path, sr=sr)
        y, _ = librosa.effects.trim(y)

        S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
        S_DB = librosa.power_to_db(S, ref=np.max, top_db=top_db)

        plt.figure(figsize=(4, 4), dpi=32)  # 128×128
        librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, cmap=cmap)
        plt.axis('off')
        plt.tight_layout(pad=0)
        plt.savefig(image_save_path, bbox_inches='tight', pad_inches=0)
        plt.close()
    except Exception as e:
        print(f"[SKIPPED] Failed to create spectrogram for {audio_path}: {e}")


In [ ]:
def extract_audio_features_to_csv(audio_dir, image_dir, output_csv):
    os.makedirs(image_dir, exist_ok=True)
    data = []

    for genre in os.listdir(audio_dir):
        genre_path = os.path.join(audio_dir, genre)
        image_genre_path = os.path.join(image_dir, genre)
        os.makedirs(image_genre_path, exist_ok=True)

        for filename in os.listdir(genre_path):
            if not filename.endswith('.wav'):
                continue

            audio_path = os.path.join(genre_path, filename)
            image_path = os.path.join(image_genre_path, filename.replace('.wav', '.png'))

            try:
                y, sr = librosa.load(audio_path, sr=22050)
                y, _ = librosa.effects.trim(y)

                #generate_mel_spectrogram(audio_path, image_path)

                features = {
                    "filename": filename,
                    "label": genre,
                    "length": librosa.get_duration(y=y, sr=sr),
                    "chroma_stft_mean": np.mean(librosa.feature.chroma_stft(y=y, sr=sr)),
                    "chroma_stft_var": np.var(librosa.feature.chroma_stft(y=y, sr=sr)),
                    "rms_mean": np.mean(librosa.feature.rms(y=y)),
                    "rms_var": np.var(librosa.feature.rms(y=y)),
                    "spectral_centroid_mean": np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
                    "spectral_centroid_var": np.var(librosa.feature.spectral_centroid(y=y, sr=sr)),
                    "spectral_bandwidth_mean": np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)),
                    "spectral_bandwidth_var": np.var(librosa.feature.spectral_bandwidth(y=y, sr=sr)),
                    "rolloff_mean": np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)),
                    "rolloff_var": np.var(librosa.feature.spectral_rolloff(y=y, sr=sr)),
                    "zero_crossing_rate_mean": np.mean(librosa.feature.zero_crossing_rate(y)),
                    "zero_crossing_rate_var": np.var(librosa.feature.zero_crossing_rate(y)),
                    "harmony_mean": np.mean(librosa.effects.harmonic(y)),
                    "harmony_var": np.var(librosa.effects.harmonic(y)),
                    "perceptr_mean": np.mean(librosa.effects.percussive(y)),
                    "perceptr_var": np.var(librosa.effects.percussive(y)),
                    "tempo": librosa.feature.rhythm.tempo(y=y, sr=sr)[0]
                }

                mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
                for i in range(1, 21):
                    features[f"mfcc{i}_mean"] = np.mean(mfccs[i-1])
                    features[f"mfcc{i}_var"] = np.var(mfccs[i-1])

                data.append(features)

            except Exception as e:
                print(f"[SKIPPED] {audio_path} — could not load/process: {e}")
                continue

    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Done! Saved {len(df)} entries to: {output_csv}")

In [22]:
extract_audio_features_to_csv(
    audio_dir=raw_audio_dir,
    image_dir= spectrogram_dir,
    output_csv= csv_dir
)


C:\Users\ebube\AppData\Local\Temp\ipykernel_8812\327411779.py:18: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=22050)
d:\Python\Python312\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


[SKIPPED] E:\Music Classifier\Scratch\Data\genres_original\jazz\jazz.00054.wav — could not load/process: 

✅ Done! Saved 999 entries to: E:\Music Classifier\Scratch\Data\features.csv
